In [28]:
import pickle
import sys
import typing
from pathlib import Path


from itertools import combinations

import numpy as np
import pandas as pd
import scipy
import yaml
from data.jaad.jaad_data import JAAD
from numpy.lib.stride_tricks import sliding_window_view
from scipy import signal
from tqdm import tqdm
from multiprocessing.pool import ThreadPool as Pool
from tqdm.contrib.concurrent import process_map 

from src.data_utils import JaadDatabase

Function running with parameter regen: False
Loading saved file from data/processed/sample_sequence_dict.pkl


In [31]:

l_

[(102, 10, 0),
 (102, 10, 1),
 (102, 10, 2),
 (102, 10, 3),
 (102, 10, 4),
 (102, 10, 5),
 (102, 10, 6),
 (102, 10, 7),
 (102, 10, 8),
 (102, 10, 9),
 (102, 10, 10),
 (102, 10, 11),
 (102, 10, 12),
 (102, 10, 13),
 (102, 10, 14),
 (102, 11, 0),
 (102, 11, 1),
 (102, 11, 2),
 (102, 11, 3),
 (102, 11, 4),
 (102, 11, 5),
 (102, 11, 6),
 (102, 11, 7),
 (102, 11, 8),
 (102, 11, 9),
 (102, 11, 10),
 (102, 11, 11),
 (102, 11, 12),
 (102, 11, 13),
 (102, 11, 14),
 (102, 12, 0),
 (102, 12, 1),
 (102, 12, 2),
 (102, 12, 3),
 (102, 12, 4),
 (102, 12, 5),
 (102, 12, 6),
 (102, 12, 7),
 (102, 12, 8),
 (102, 12, 9),
 (102, 12, 10),
 (102, 12, 11),
 (102, 12, 12),
 (102, 12, 13),
 (102, 12, 14),
 (102, 13, 0),
 (102, 13, 1),
 (102, 13, 2),
 (102, 13, 3),
 (102, 13, 4),
 (102, 13, 5),
 (102, 13, 6),
 (102, 13, 7),
 (102, 13, 8),
 (102, 13, 9),
 (102, 13, 10),
 (102, 13, 11),
 (102, 13, 12),
 (102, 13, 13),
 (102, 14, 0),
 (102, 14, 1),
 (102, 14, 2),
 (102, 14, 3),
 (102, 14, 4),
 (102, 14, 5),
 (102,

In [30]:

for (i, j, k) in l_:
    try:
        b = BodyBuilder(data[i][j]['skeleton_sequence'][k])
        b.length_body
    except:
        print(f"{i}, {j}, {k}")

In [32]:
b = BodyBuilder(data[353][41]['skeleton_sequence'][7])

In [39]:
data[174].__len__()

194